In [1]:
import os

os.chdir('/content/')
!git clone https://github.com/guillermoiglesiashernandez/StyleGAN
os.chdir('/content/StyleGAN/')

fatal: destination path 'StyleGAN' already exists and is not an empty directory.


In [2]:
!pip3 install tensorflow_addons

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from train.train import train
from train.style_gan import StyleGAN
from train.plot_images import plot_images
from data_loader.data_download import download_data

In [4]:
style_gan = StyleGAN(start_res=4, target_res=16)

In [5]:
url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
output_dir = "celeba_gan"

data = download_data(url, output_dir)

Found 202599 files belonging to 1 classes.


In [6]:
train(style_gan, data, start_res=4, target_res=16, steps_per_epoch=1, display_images=False)

"""
## Results
We can now run some inference using pre-trained 64x64 checkpoints. In general, the image
fidelity increases with the resolution. You can try to train this StyleGAN to resolutions
above 128x128 with the CelebA HQ dataset.
"""

url = "https://github.com/soon-yau/stylegan_keras/releases/download/keras_example_v1.0/stylegan_128x128.ckpt.zip"

weights_path = keras.utils.get_file(
    "stylegan_128x128.ckpt.zip",
    url,
    extract=True,
    cache_dir=os.path.abspath("."),
    cache_subdir="pretrained",
)

style_gan.grow_model(128)
style_gan.load_weights(os.path.join("pretrained/stylegan_128x128.ckpt"))

tf.random.set_seed(196)
batch_size = 2
z = tf.random.normal((batch_size, style_gan.z_dim))
w = style_gan.mapping(z)
noise = style_gan.generate_noise(batch_size=batch_size)
images = style_gan({"style_code": w, "noise": noise, "alpha": 1.0})
plot_images(images, 5)

"""
## Style Mixing
We can also mix styles from two images to create a new image.
"""

alpha = 0.4
w_mix = np.expand_dims(alpha * w[0] + (1 - alpha) * w[1], 0)
noise_a = [np.expand_dims(n[0], 0) for n in noise]
mix_images = style_gan({"style_code": w_mix, "noise": noise_a})
image_row = np.hstack([images[0], images[1], mix_images[0]])
plt.figure(figsize=(9, 3))
plt.imshow(image_row)
plt.axis("off")


Model resolution:4x4
STABLE
1/1 [==============================] - 10s 10s/step - d_loss: 4.7932 - g_loss: -0.6010

Model resolution:8x8
TRANSITION


KeyboardInterrupt: ignored